# Demo filter

A demonstration of the edge filter with controllable strength.

In [21]:
import numpy as np
import cv2
from scipy.signal import convolve2d
import math, time

# ---- Filters ----
def edge_filter(im, intensity):
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY).astype(np.float32)
    weight_edge_hor = np.array([[-1,0,1],
                                [-2,0,2],
                                [-1,0,1]])
    weight_edge_ver = np.array([[-1,-2,-1],
                                [ 0, 0, 0],
                                [ 1, 2, 1]])
    im_edge_ver = convolve2d(im_gray, weight_edge_hor, mode='same', boundary='symm')
    im_edge_hor = convolve2d(im_gray, weight_edge_ver, mode='same', boundary='symm')
    im_edge = np.sqrt(im_edge_hor**2 + im_edge_ver**2)
    im_edge = np.clip(im_edge * intensity, 0, 255).astype(np.uint8)
    edges_colored = cv2.cvtColor(im_edge, cv2.COLOR_GRAY2BGR)
    return cv2.addWeighted(im, 0.8, edges_colored, 0.6, 0)

def blur_filter(im, intensity):
    k = int(max(1, intensity * 3))
    if k % 2 == 0:
        k += 1
    return cv2.GaussianBlur(im, (k, k), 0)

def wave_filter(im, intensity):
    rows, cols, _ = im.shape
    im_out = np.zeros_like(im)
    t = time.time()
    for i in range(rows):
        shift_x = int(6 * intensity * math.sin(2 * math.pi * i / 120 + t))
        im_out[i] = np.roll(im[i], shift_x, axis=0)
    return im_out

# ---- Overlay ----
def draw_overlay(im, effects):
    h, w, _ = im.shape
    overlay = np.zeros((40, w, 3), dtype=np.uint8)
    cv2.rectangle(overlay, (0, 0), (w, 40), (0, 0, 0), -1)
    text = " | ".join(
        [f"{k.upper()}:{'ON' if v['on'] else 'OFF'} ({v['intensity']:.1f})"
         for k, v in effects.items()]
    )
    cv2.putText(overlay, text, (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    return np.vstack([overlay, im])

# ---- Main ----
cv2.startWindowThread()
camera = cv2.VideoCapture(0)

effects = {
    'edge': {'on': False, 'intensity': 1.0},
    'blur': {'on': False, 'intensity': 1.0},
    'wave': {'on': False, 'intensity': 1.0}
}
last_active = None

print("Controls:")
print(" A - toggle Edge filter")
print(" B - toggle Blur filter")
print(" C - toggle Wave filter")
print(" ↑ / ↓ - adjust intensity of last toggled filter")
print(" Q - quit")

try:
    while True:
        ret, im = camera.read()
        if not ret:
            break

        # Apply all active filters in sequence
        if effects['edge']['on']:
            im = edge_filter(im, effects['edge']['intensity'])
        if effects['blur']['on']:
            im = blur_filter(im, effects['blur']['intensity'])
        if effects['wave']['on']:
            im = wave_filter(im, effects['wave']['intensity'])

        im_display = draw_overlay(im, effects)
        cv2.imshow("Enric's CrazyCam", im_display)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('a'):
            effects['edge']['on'] = not effects['edge']['on']
            last_active = 'edge'
            print(f'Edge {"ON" if effects["edge"]["on"] else "OFF"}')

        elif key == ord('b'):
            effects['blur']['on'] = not effects['blur']['on']
            last_active = 'blur'
            print(f'Blur {"ON" if effects["blur"]["on"] else "OFF"}')

        elif key == ord('c'):
            effects['wave']['on'] = not effects['wave']['on']
            last_active = 'wave'
            print(f'Wave {"ON" if effects["wave"]["on"] else "OFF"}')

        elif key == 82:  # Up arrow
            if last_active:
                effects[last_active]['intensity'] = min(5.0, effects[last_active]['intensity'] + 0.2)
                print(f'{last_active} intensity: {effects[last_active]["intensity"]:.1f}')

        elif key == 84:  # Down arrow
            if last_active:
                effects[last_active]['intensity'] = max(0.1, effects[last_active]['intensity'] - 0.2)
                print(f'{last_active} intensity: {effects[last_active]["intensity"]:.1f}')

        elif key == ord('q'):
            break

finally:
    camera.release()
    cv2.destroyAllWindows()
    for _ in range(4):
        cv2.waitKey(1)


Controls:
 A - toggle Edge filter
 B - toggle Blur filter
 C - toggle Wave filter
 ↑ / ↓ - adjust intensity of last toggled filter
 Q - quit
Wave ON
wave intensity: 1.2
wave intensity: 1.4
wave intensity: 1.6
wave intensity: 1.8
wave intensity: 2.0
wave intensity: 2.2
wave intensity: 2.4
Edge ON
Blur ON
Blur OFF
Edge OFF
Wave OFF
Wave ON
Wave OFF
Wave ON
Wave OFF
